In [ ]:
#Installing required package to run the Notebook (not needed if install.sh was executed)
#!pip install exspy
#!pip install exspy --upgrade
#!pip install numba
#!pip install hyperspy --upgrade
#!python3.10 -m ipykernel install --user
!pip install openpyxl

In [3]:
import os;
import numpy as np;
import matplotlib.pyplot as plt;
from openpyxl import load_workbook;
from scipy.spatial import QhullError;
from scipy.interpolate import griddata;
from ipywidgets import widgets, Layout, GridBox;
from IPython.display import display;

W_folderpath = widgets.Dropdown(
    options=[(elm, elm) for elm in os.listdir('./data/EDX/')
                                         if not elm.startswith('~$') and elm.endswith('.xlsx')],
    value=None,
    description='Folder',
    disabled=False,
    layout=Layout(width='auto', grid_area='path')
);
W_elm = widgets.Dropdown(
    options=[('Select a folder', None)],
    value=None,
    description='Element',
    disabled=False,
    layout=Layout(width='100%', grid_area='elm')
);
W_EDX_out = widgets.Output(
    layout=Layout(width='auto', grid_area='plot')
);

#interface layout and box
EDX_plot_layout = Layout(
    width='100%',
    grid_gap='0px 0px',
    grid_template_rows='auto auto',
    grid_template_columns='16.66% 16.66% 16.66% 16.66% 16.66% 16.66%',
    grid_template_areas='''
    "path path elm elm . ."
    "plot plot plot plot plot plot"
    '''
);
EDX_gridbox = GridBox(
    children=[W_folderpath, W_elm, W_EDX_out],
    layout=EDX_plot_layout);

def EDX_plot(change):
    with W_EDX_out:
        #clear the previous plot
        W_EDX_out.clear_output();
        
        #loading the data inside the .xlsx file
        datafile = f'./data/EDX/{W_folderpath.value}';
        try:
            wb = load_workbook(filename = datafile);
        except FileNotFoundError:
            print("Not found: Error code 1");
            return 1;
        ws = wb.active;

        #putting all the data inside lists to obtain a treatable format
        LIST_DATA = []; X_POS = []; Y_POS = []; ELM = [];
        for i, row in enumerate(ws.values):
            LIST_DATA.append(row);
            
        #checking all the element that have been quantified by excluding the deconvoluted ones
        index_tab = [];
        for index, elm in enumerate(LIST_DATA[-3]):
            if index > 0 and elm is not None:
                if float(elm) != 0.0:
                    index_tab.append(index);
        elm_options = [(LIST_DATA[0][index], index) for index in index_tab];
        if elm_options != list(W_elm.options):
            W_elm.options = elm_options;
            return 0;
        index = W_elm.options[0][-1];
        elm_value = W_elm.value;

        #change index accordingly to prompt from user
        if index != elm_value:
            index = elm_value;
        
        #reading the data for plotting
        step_x, step_y = 5, 5;
        start_x, start_y = -40, -40;
        for row in LIST_DATA:
            if row[0] == 'Spectrum':
                title = row[index];
            elif row[0].startswith('Spectrum_'):
                x_index, y_index = row[0].split('(')[-1].split(')')[0].split(',');
                x_pos, y_pos = (int(x_index)-1)*step_x+start_x, (int(y_index)-1)*step_y+start_y;
                if np.abs(x_pos) + np.abs(y_pos) <= 60 :
                    X_POS.append(x_pos); Y_POS.append(y_pos);
                    ELM.append(float(row[index]));
            else:
                #print(row[0]);
                continue;

        #create x-y points to be used in heatmap
        xi = np.linspace(np.min(X_POS), np.max(X_POS), 500);
        yi = np.linspace(np.min(Y_POS), np.max(Y_POS), 500);
                        
        #interpolation between the points for 2D map
        try:
            zi = griddata((X_POS, Y_POS), ELM, (xi[None,:], yi[:,None]), method='cubic');
        except QhullError:
            print(f"Error: Data is not 2-dimensionnal, cannot create a map");
            return 1;
        # Create the contour plot
        plt.figure(figsize=(7.5, 6));
        plt.title(f'{title} (in at.%)');
        CS = plt.contourf(xi, yi, zi, 100, cmap=plt.cm.rainbow, vmax=max(ELM), vmin=min(ELM));
        plt.colorbar(CS);
        plt.figure(dpi=400);
        plt.show();
    return 0;

display(EDX_gridbox);
W_elm.observe(EDX_plot, names='value');
W_folderpath.observe(EDX_plot, names='value');

GridBox(children=(Dropdown(description='Folder', layout=Layout(grid_area='path', width='auto'), options=(('291…

In [ ]:
#attempt at doing element quantification with EDX spectra (not working)
import exspy;
import hyperspy.api as hs;
import numpy as np;
#from hyperspy import quantification;

#reading spectrum.txt file
with open('./data/EDX/DyCo-Pd_1-6/Spectrum_(11,1).txt') as file:
    for line in file:
        if line.startswith("Primary energy"):
            beam_energy = float(line.split(': ')[1].strip());
        elif line.startswith("Tilt angle"):
            tilt_stage = float(line.split(': ')[1].strip());
        elif line.startswith("Azimut angle"):
            azimuth_angle = float(line.split(': ')[1].strip());
        elif line.startswith("Take off angle"):
            elevation_angle = float(line.split(': ')[1].strip());
        elif line.startswith("Mn FWHM"):
            energy_resolution_MnKa = float(line.split(': ')[1].strip());
        elif line.startswith("Calibration, abs."):
            offset = float(line.split(': ')[1].strip())/1000;
        elif line.startswith("Calibration, lin."):
            scale = float(line.split(': ')[1].strip())/1000;
            
        elif line.startswith("Energy Counts"):
            data_tab = [int(y) for y in file.read().split()[1::2]];
            break;

#add data inside a signal1D
s = hs.signals.Signal1D(data_tab);

#set EDX metadata parameters
s.set_signal_type('EDS_SEM');
s.metadata.General.title = 'DyCo-Pd_1-6';
s.set_microscope_parameters(beam_energy = beam_energy,
                            tilt_stage=tilt_stage,
                            azimuth_angle=azimuth_angle,
                            elevation_angle=elevation_angle,
                            energy_resolution_MnKa= energy_resolution_MnKa);
#set energy axes and scale
s.axes_manager[-1].name = 'E';
s.axes_manager['E'].units = 'keV';
s.axes_manager['E'].scale = scale;
s.axes_manager['E'].offset = offset;

#definie element to characterize
s.set_elements(['Si', 'Co', 'Dy','Pd', 'C', 'O']);
s.add_lines();

#display stuff
print(s.get_lines_intensity(plot_result=True));
print("---")
#background substraction and intensities calculations
bw = s.estimate_background_windows(line_width=[5.0, 1.0]);
m = s.create_model();
m.fix_background();

m.fit();
m.fit_background(start_energy=2.0);

intensities = m.get_lines_intensity(background_windows=bw, plot_result=True);
m.plot(navigator='slider', autoscale='x');

#print(hs.quantification);